In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing import image

from collections import defaultdict

import time
import os
from PIL import Image
#Hash images using the ImageHash library
#https://pypi.python.org/pypi/ImageHash
import imagehash

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
time.time()

1517720559.493251

In [3]:
time.time()

1517720566.3615081

In [4]:
type(time.time())

float

In [5]:
lst = ['a', 'b', 'c']
lst[0:1]

['a']

In [ ]:
d = defaultdict(list, [(a, b) for a,b in zip(lst, )])

In [ ]:
manager = ImageManager('~/tree-logic/tree_photos/')
directory_names = ['picea_sitchensis', 'betula_papyrifera']
#manager.take_inventory(directory_names) #
#or manager.take_inventory() #default is to walk through all subdirectories
manager.sync_images(directory_names) #Syncs the manager's dataframe and dictionary to the specified directories
#or manager.sync_images() #default is to sync all subdirectories

manager.get_duplicates() #get dictionary of image hashes mapping to more than one path
manager.remove_duplicates() #delete duplicate image files